In [1]:
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import torch
import os
from data_casual import output_list_train, input_list_train, input_list_test, output_list_test
from eval import extracting_steering_vector, calc_loss_steering_vector

ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"
import scipy.stats as stats

In [2]:
import math

In [2]:
generator = Llama.build(ckpt_dir=ckpt_dir, tokenizer_path=tokenizer_path, max_seq_len= 1024, max_batch_size= 4, activation=True, activation_layer=12)
print(generator)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


c:\Users\Ribbe\Coding\VSC\FU_Berlin\Actual_Work\llama3\llama\generation.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=

Loaded in 151.64 seconds


In [3]:
temperature = 0.6
top_p = 0.9
max_seq_len = 1024
max_batch_size = 4
max_gen_len= 1

In [4]:
input_list_train[12]

[{'role': 'system',
  'content': 'Answer with T if you believe the following statement is correct, answer with F if you believe it is false, do not give any reasoning.'},
 {'role': 'system',
  'content': 'You are a helpful assistant in casual reasoning, is the following reasoning true?'},
 {'role': 'user',
  'content': 'Premise: Suppose there is a closed system of 4 variables, A, B, C and D. All the statistical relations among these 4 variables are as follows: A correlates with C. A correlates with D. B correlates with C. B correlates with D. C correlates with D. However, A is independent of B. A and D are independent given B and C. A and D are independent given C. B and D are independent given A and C. B and D are independent given C.\nHypothesis: A directly causes C.'}]

In [24]:
#checking the random baseline
average_loss = 0
for k in range(1000):
    p = output_list_test.count("1")/len(input_list_test)
    x = stats.bernoulli.rvs(p, size = len(input_list_test))
    loss = 1/len(input_list_test)*sum([abs(int(x[i])-int(k)) for i, k in enumerate(output_list_test)])
    average_loss += 1/1000*loss
print(average_loss, p)

0.26221170395869187 0.1549053356282272


In [5]:
#extracting a steering vector for every layer
average_loss = 0
for k in range(10,32):
    steering_vector, loss = extracting_steering_vector(generator, (input_list_train, output_list_train),layer=k, iter=2000)
    steering_vec = torch.nn.functional.normalize(steering_vector, dim=0)
    torch.save(steering_vector, f"./vectors_last_token/vector{k}.pt")
    print(f"Training loss is {loss}")
    average_loss += 1/len(generator.model.layers)*loss
print(f"Average loss is {average_loss}")

c:\Users\Ribbe\Coding\VSC\FU_Berlin\Actual_Work\llama3\eval\eval.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  instruct_vec = torch.load(f"./steering_vectors/instruct

Training loss is 0.006396609213839021
Training loss is 0.006454936957430255
Training loss is 0.006377166632641943
Training loss is 0.006352863406145596
Training loss is 0.0065132647010214895
Training loss is 0.006425773085634638
Training loss is 0.006459797602729525
Training loss is 0.00631397824375144
Training loss is 0.006348002760846326
Training loss is 0.006425773085634638
Training loss is 0.006377166632641943
Training loss is 0.006299396307853631
Training loss is 0.006377166632641943
Training loss is 0.0062653717907587446
Training loss is 0.006348002760846326
Training loss is 0.006440355021532447
Training loss is 0.006382027277941213
Training loss is 0.006435494376233177
Training loss is 0.006401469859138291


KeyboardInterrupt: 

In [7]:
#analysing the steering vectors
for k in range(12, 32):
    st_vec = torch.load(f"./vectors_last_token/vector{k}.pt")
    print(torch.norm(st_vec, dim=0), torch.mean(st_vec, dim=0)) 

C:\Users\Ribbe\AppData\Local\Temp\ipykernel_22676\270925784.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  st_vec = torch.load(f"./vectors_last_token/vector{k}.pt")


tensor(2.9219) tensor(-0.0004)
tensor(2.9844) tensor(-0.0003)
tensor(3.1562) tensor(6.7711e-05)
tensor(3.1875) tensor(-6.7711e-05)
tensor(3.3125) tensor(-0.0002)
tensor(3.2812) tensor(-3.6955e-05)
tensor(3.3281) tensor(-0.0008)
tensor(3.5781) tensor(-0.0006)
tensor(3.7344) tensor(-0.0009)
tensor(3.7656) tensor(-0.0005)
tensor(4.6250) tensor(0.0005)
tensor(4.1875) tensor(-0.0006)
tensor(4.6250) tensor(-0.0002)
tensor(5.5938) tensor(-0.0010)
tensor(6.0312) tensor(-0.0014)
tensor(6.7500) tensor(-0.0016)
tensor(7.4062) tensor(-0.0003)


FileNotFoundError: [Errno 2] No such file or directory: './vectors_last_token/vector29.pt'

In [ ]:
#calculating the loss for each layes
#initial_loss = calc_loss_steering_vector(generator, torch.tensor(4096*[0]), (input_list_test, output_list_test), layer=12, iter=len(input_list_test), multiplier=0)
losses_last_token_layer = {}
print(f"initial loss is {0.68}")
for k in range(12,32):
    steering_vec = torch.load(f"./vectors_last_token/vector{k}.pt")
    for multiplier in range(1,25,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses_last_token_layer[f"{k},{multiplier}"] = loss
        if loss>0.85:
            break

initial loss is 0.68


C:\Users\Ribbe\AppData\Local\Temp\ipykernel_15724\2526441504.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load(f"./vectors_last_token/vector{k}.p

0 questions were wrongly classified
Layer 12 loss with multiplier 1 is 0.6712564543889873
7 questions were wrongly classified
Layer 12 loss with multiplier 3 is 0.36574870912219953
1162 questions were wrongly classified
Layer 12 loss with multiplier 5 is 1.0000000000000189
0 questions were wrongly classified
Layer 13 loss with multiplier 1 is 0.4836488812392373
0 questions were wrongly classified
Layer 13 loss with multiplier 3 is 0.15490533562822684
1162 questions were wrongly classified
Layer 13 loss with multiplier 5 is 1.0000000000000189
0 questions were wrongly classified
Layer 14 loss with multiplier 1 is 0.5154905335628178
0 questions were wrongly classified
Layer 14 loss with multiplier 3 is 0.2641996557659188
1162 questions were wrongly classified
Layer 14 loss with multiplier 5 is 1.0000000000000189
0 questions were wrongly classified
Layer 15 loss with multiplier 1 is 0.7220309810671309
0 questions were wrongly classified
Layer 15 loss with multiplier 3 is 0.2865748709122179

In [5]:
#calculating the loss for each layes
#initial_loss = calc_loss_steering_vector(generator, torch.tensor(4096*[0]), (input_list_test, output_list_test), layer=12, iter=len(input_list_test), multiplier=0)
losses_last_token_layer = {}
print(f"initial loss is {0.68}")
for k in range(18,32):
    steering_vec = torch.load(f"./vectors_last_token/vector{k}.pt")
    for multiplier in range(1,25,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses_last_token_layer[f"{k},{multiplier}"] = loss
        if loss>0.85:
            break

initial loss is 0.68


C:\Users\Ribbe\AppData\Local\Temp\ipykernel_22676\1496298509.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load(f"./vectors_last_token/vector{k}.p

0 questions were wrongly classified
Layer 18 loss with multiplier 1 is 0.6419965576592096
0 questions were wrongly classified
Layer 18 loss with multiplier 3 is 0.8055077452667908
0 questions were wrongly classified
Layer 18 loss with multiplier 5 is 0.8433734939759149
0 questions were wrongly classified
Layer 18 loss with multiplier 7 is 0.7426850258175622
1162 questions were wrongly classified
Layer 18 loss with multiplier 9 is 1.0000000000000189
0 questions were wrongly classified
Layer 19 loss with multiplier 1 is 0.6876075731497454
0 questions were wrongly classified
Layer 19 loss with multiplier 3 is 0.7960413080895098
0 questions were wrongly classified
Layer 19 loss with multiplier 5 is 0.8442340791738495
0 questions were wrongly classified
Layer 19 loss with multiplier 7 is 0.8450946643717842
390 questions were wrongly classified
Layer 19 loss with multiplier 9 is 0.8958691910499278
0 questions were wrongly classified
Layer 20 loss with multiplier 1 is 0.7177280550774577
0 que

KeyboardInterrupt: 

In [5]:
for k in range(10,32):
    vec = torch.load(f"./steering_without_instruct/vector{k}.pt")
    vec = torch.nn.functional.normalize(vec, dim=0)
    torch.save(vec, f"./steering_without_instruct/n_vector{k}.pt")

C:\Users\Ribbe\AppData\Local\Temp\ipykernel_15880\4185949204.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vec = torch.load(f"./steering_without_instruct/vector{k}.pt"

In [25]:
losses2 = {}
initial_loss = 0.68
len(input_list_test)

1162

In [35]:
norms = []
for k in range(32):
    v = torch.load(f"./steering_without_instruct/vector{k}.pt")
    norm = torch.norm(v)
    norm = math.ceil(norm.item())
    norms.append(norm)

print(norms)


C:\Users\Ribbe\AppData\Local\Temp\ipykernel_3580\158326343.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  v = torch.load(f"./steering_without_instruct/vector{k}.pt")


[1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 8, 8, 8, 9, 10, 10, 11, 12, 18]


In [38]:
#calculating the loss for each layes
#initial_loss = calc_loss_steering_vector(generator, torch.tensor(4096*[0]), (input_list_test, output_list_test), layer=12, iter=len(input_list_test), multiplier=0)
print(f"initial loss is {initial_loss}")
for k in range(16,32):
    steering_vec = torch.load(f"./c_and_n_vectors/vector{k}.pt")
    for multiplier in range(1,40,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses2[f"{k},{multiplier}"] = loss
        if loss>0.9:
            break

initial loss is 0.68


C:\Users\Ribbe\AppData\Local\Temp\ipykernel_3580\2461276629.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load(f"./c_and_n_vectors/vector{k}.pt")


Layer 16 loss with multiplier 1 is 0.660068846815837
Layer 16 loss with multiplier 3 is 0.5955249569707391
Layer 16 loss with multiplier 5 is 0.5481927710843341
Layer 16 loss with multiplier 7 is 0.44836488812391945
Layer 16 loss with multiplier 9 is 0.28227194492254504
Layer 16 loss with multiplier 11 is 0.9991394148020843
Layer 17 loss with multiplier 1 is 0.7125645438898499
Layer 17 loss with multiplier 3 is 0.809810671256464
Layer 17 loss with multiplier 5 is 0.8390705679862417
Layer 17 loss with multiplier 7 is 0.8450946643717842
Layer 17 loss with multiplier 9 is 0.8433734939759149
Layer 17 loss with multiplier 11 is 0.8950086058519932
Layer 17 loss with multiplier 13 is 0.9991394148020843
Layer 18 loss with multiplier 1 is 0.7512908777969086
Layer 18 loss with multiplier 3 is 0.8261617900172221
Layer 18 loss with multiplier 5 is 0.838209982788307
Layer 18 loss with multiplier 7 is 0.8347676419965685
Layer 18 loss with multiplier 9 is 0.8046471600688562
Layer 18 loss with multipl

In [ ]:
import json
with open("initial_values2.json", "w") as file:
    json.dump(losses2, file)

[tensor([-0.0361, -0.0046, -0.0242,  ..., -0.0271, -0.0303,  0.0280])]

In [11]:
vec1 = torch.load("./steering_vectors/vector2.pt")
vec2 = torch.load("./steering_without_instruct/vector2.pt")

C:\Users\Ribbe\AppData\Local\Temp\ipykernel_3580\3856237667.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vec1 = torch.load("./steering_vectors/vector2.pt")
C:\Users\R

In [22]:
for k in range(32):
    vec = torch.load(f"./steering_without_instruct/vector{k}.pt")
    c_vec = vec - torch.mean(vec)
    n_c_vec = torch.nn.functional.normalize(c_vec, dim=0)
    torch.save(n_c_vec, f"./c_and_n_vectors/vector{k}.pt")


C:\Users\Ribbe\AppData\Local\Temp\ipykernel_3580\3787317746.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vec = torch.load(f"./steering_without_instruct/vector{k}.pt")

In [5]:
#seeing if the general reasoning abilities improve
steering_vec = 14.5*torch.load("./steering_without_instruct/n_vector26.pt")


C:\Users\Ribbe\AppData\Local\Temp\ipykernel_24320\685555240.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = 14.5*torch.load("./steering_without_instruct/n

In [ ]:
generator.change_activation_vector(steering_vec)
generator.change_activation_layer(26)
value, activations = generator.chat_completion([input_list_test[1]],
                                   max_gen_len=max_gen_len,
                                   top_p=top_p,
                                   temperature=temperature)

In [27]:
value

[{'generation': {'role': 'assistant',
   'content': 'F.\n\nThe given statistical relations among A, B, C, D, and E suggest a complex network of correlations and independence. However, the hypothesis that B causes something else which causes D is not supported by the given information.\n\nA and D are independent given B and C, and also given B, C, and E. This suggests that A and D are not directly related, and that B and C, or B, C, and E, are sufficient to explain the independence of A and D.\n\nFurthermore, D and E are independent given A, B, and C, and also given A and C. This suggests that D and E are not directly related, and that A, B, and C, or A and C, are sufficient to explain the independence of D and E.\n\nThere is no information that suggests B causes something else which causes D. In fact, B and E are independent given A and C, and also given A, C, and D. This suggests that B and E are not directly related, and that A and C, or A, C, and D, are sufficient to explain the ind

In [5]:
negative_indices = [i for i,x in enumerate(output_list_test) if x=="0"]
positive_indices = [i for i,x in enumerate(output_list_test) if x=="1"]

In [6]:
negative_indices_train =  [i for i,x in enumerate(output_list_train) if x=="0" and i<=1094]
positive_indices_train =  [i for i,x in enumerate(output_list_train) if x=="1" and i<=7865]

In [7]:

input_positive_list_train = [input_list_train[k] for k in positive_indices_train]
input_negative_list_train = [input_list_train[k] for k in negative_indices_train]
input_list_train_short = input_positive_list_train + input_negative_list_train
output_list_train_short = 1000*["1"] + 1000*["0"]
print(len(input_list_train_short) == len(output_list_train_short))

True


In [8]:
#extracting a steering vector for every layer while keeping the amount of positive and negative vectors constant
average_loss = 0
for k in range(32):
    steering_vec, loss = extracting_steering_vector(generator, (input_list_train_short, output_list_train_short),layer=k, iter=2000)
    steering_vec = torch.nn.functional.normalize(steering_vec, dim=0)
    torch.save(steering_vec, f"./even_vectors/vector{k}.pt")
    print(f"Training loss is {loss}")
    average_loss += 1/len(generator.model.layers)*loss
print(f"Average loss is {average_loss}")

Training loss is 0.5539999999999944
Training loss is 0.5534999999999944
Training loss is 0.548499999999995
Training loss is 0.5509999999999947
Training loss is 0.5489999999999949
Training loss is 0.5624999999999934
Training loss is 0.5504999999999948
Training loss is 0.5599999999999937
Training loss is 0.5524999999999946
Training loss is 0.5534999999999944
Training loss is 0.5489999999999949
Training loss is 0.5549999999999943
Training loss is 0.557999999999994
Training loss is 0.5499999999999948
Training loss is 0.5604999999999937
Training loss is 0.5519999999999946
Training loss is 0.557499999999994
Training loss is 0.5524999999999946
Training loss is 0.5474999999999951
Training loss is 0.5489999999999949
Training loss is 0.5619999999999935
Training loss is 0.5554999999999942
Training loss is 0.5529999999999945
Training loss is 0.5534999999999944
Training loss is 0.547999999999995
Training loss is 0.5524999999999946
Training loss is 0.5499999999999948
Training loss is 0.5519999999999

In [24]:
generator.change_activation_vector(torch.tensor(4096*[0]))

In [33]:
input_list_train_short[1203]

[{'role': 'system',
  'content': 'Answer with T if you believe the following statement is correct, answer with F if you believe it is false, do not give any reasoning.'},
 {'role': 'system',
  'content': 'You are a helpful assistant in casual reasoning, is the following reasoning true?'},
 {'role': 'user',
  'content': 'Premise: Suppose there is a closed system of 4 variables, A, B, C and D. All the statistical relations among these 4 variables are as follows: A correlates with C. A correlates with D. B correlates with C. C correlates with D. However, A is independent of B. A and B are independent given D. B is independent of D. B and D are independent given A. B and D are independent given A and C. C and D are independent given A. C and D are independent given A and B.\nHypothesis: D directly causes B.'}]

In [39]:
value, vec = generator.chat_completion([input_list_train_short[1203]], temperature=temperature, top_p=top_p, max_gen_len=1023)
output_list_train_short[1203]

0

In [10]:
losses2 = {}

In [11]:
#calculating the loss for each layes
#initial_loss = calc_loss_steering_vector(generator, torch.tensor(4096*[0]), (input_list_test, output_list_test), layer=12, iter=len(input_list_test), multiplier=0)
print(f"initial loss is {0.68}")
for k in range(16,32):
    steering_vec = torch.load(f"./even_vectors/vector{k}.pt")
    for multiplier in range(1,25,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test, output_list_test), layer=k, iter=len(input_list_test), multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses2[f"{k},{multiplier}"] = loss
        if loss>0.85:
            break

initial loss is 0.68


C:\Users\Ribbe\AppData\Local\Temp\ipykernel_18468\965858657.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load(f"./even_vectors/vector{k}.pt")


Layer 16 loss with multiplier 1 is 0.6936316695352879
Layer 16 loss with multiplier 3 is 0.650602409638556
Layer 16 loss with multiplier 5 is 0.6058519793459548
Layer 16 loss with multiplier 7 is 0.5895008605851967
Layer 16 loss with multiplier 9 is 0.6635111876075755
Layer 16 loss with multiplier 11 is 0.7538726333907125
Layer 16 loss with multiplier 13 is 0.8080895008605947
Layer 16 loss with multiplier 15 is 0.8218588640275489
Layer 16 loss with multiplier 17 is 0.9251290877797055
Layer 17 loss with multiplier 1 is 0.6712564543889873
Layer 17 loss with multiplier 3 is 0.638554216867471
Layer 17 loss with multiplier 5 is 0.638554216867471
Layer 17 loss with multiplier 7 is 0.6626506024096409
Layer 17 loss with multiplier 9 is 0.719449225473327
Layer 17 loss with multiplier 11 is 0.7547332185886472
Layer 17 loss with multiplier 13 is 0.7934595524957059
Layer 17 loss with multiplier 15 is 0.8209982788296143
Layer 17 loss with multiplier 17 is 0.8330464716006992


KeyboardInterrupt: 

In [31]:
#even test set

negative_indices_test  = [i for i,x in enumerate(output_list_test) if x=="0" and i<184]
positive_indices_test = [i for i,x in enumerate(output_list_test) if x=="1" ]

input_positive_list_test = [input_list_test[k] for k in positive_indices_test]
input_negative_list_test = [input_list_test[k] for k in negative_indices_test]

input_list_test_short = input_positive_list_test + input_negative_list_test
output_list_test_short = 180*[1] + 180*[0]

In [32]:
for k in range(16,32):
    steering_vec = torch.load(f"./even_vectors/vector{k}.pt")
    for multiplier in range(1,25,2):
        loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test_short, output_list_test_short), layer=k, iter=360, multiplier=multiplier)
        print(f"Layer {k} loss with multiplier {multiplier} is {loss}")
        losses2[f"{k},{multiplier}"] = loss
        if loss>0.85:
            break

C:\Users\Ribbe\AppData\Local\Temp\ipykernel_18468\1235680360.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vec = torch.load(f"./even_vectors/vector{k}.pt")


Layer 16 loss with multiplier 1 is 0.3444444444444437
Layer 16 loss with multiplier 3 is 0.30833333333333274
Layer 16 loss with multiplier 5 is 0.27222222222222175
Layer 16 loss with multiplier 7 is 0.37499999999999917
Layer 16 loss with multiplier 9 is 0.4055555555555546
Layer 16 loss with multiplier 11 is 0.40277777777777685
Layer 16 loss with multiplier 13 is 0.44722222222222113
Layer 16 loss with multiplier 15 is 0.5277777777777765
Layer 16 loss with multiplier 17 is 0.7861111111111089
Layer 16 loss with multiplier 19 is 0.9194444444444417
Layer 17 loss with multiplier 1 is 0.31666666666666604
Layer 17 loss with multiplier 3 is 0.34999999999999926
Layer 17 loss with multiplier 5 is 0.30555555555555497
Layer 17 loss with multiplier 7 is 0.36111111111111033
Layer 17 loss with multiplier 9 is 0.3916666666666658
Layer 17 loss with multiplier 11 is 0.41944444444444345
Layer 17 loss with multiplier 13 is 0.42777777777777676
Layer 17 loss with multiplier 15 is 0.43611111111111006
Layer 17

KeyboardInterrupt: 

In [34]:
loss = calc_loss_steering_vector(generator, steering_vec, (input_list_test_short, output_list_test_short), layer=26, iter=360, multiplier=0)
print(f"Layer {26} loss with multiplier {0} is {loss}")

Layer 26 loss with multiplier 0 is 0.2944444444444439
